### 加载、读取

In [1]:
from PyEMD import EEMD, EMD, Visualisation
import pandas as pd
import numpy as np

In [2]:
# 设置路径
input_path = 'C:\\Users\\iii\\Desktop\\LSTM+AQI\\数据和测算结果\\'
output_path = 'C:\\Users\\iii\\Desktop\\LSTM+AQI\\数据和测算结果\\'

In [3]:
# 读取数据
data_0 = pd.read_excel(input_path+'北京+日+AQI.xlsx', index_col='日期')

In [4]:
# 展示数据
data_0.head(2)

,t,AQI,质量等级,PM2.5,PM10,SO2,NO2,CO,O3_8h
日期,,,,,,,,,
2014-01-01,1,81,良,45,111,28.0,62.0,1.5,52
2014-01-02,2,145,轻度污染,111,168,69.0,93.0,3.4,14


### 切片 取数

In [5]:
# aqi = data_0[['t','AQI']]
aqi = data_0.AQI

In [6]:
# 展示
aqi.head(3)

日期
2014-01-01     81
2014-01-02    145
2014-01-03     74
Name: AQI, dtype: int64

### 转换格式
##### 需求： <class 'numpy.ndarray'>

In [7]:
aqi_ndarray = np.asarray(aqi)
print(type(aqi))
'\n'
print(aqi_ndarray)

<class 'pandas.core.series.Series'>
[ 81 145  74 149 119 182 145  27  46  85 139  47 109 108 125 402 215  99
 151  35  26 150 271 197  42  72 118  62 155  96 190 179  89  37  40 117
 157 133  90  33  35 129 146 238 315 428 346 138  95  87 214 309 313 261
 310 403 366  35 103  54 178 268  77  35  39  66 172 124 107 205  41  62
  64 124 148 145  87  72  44  57  82 177 270 250 319 299 168  63  64 176
 137 130  65  91  65  96 128 224 159 251  81 131 207 279 120  82 120 183
  83  81  67  86 151 151 171  66  73  84 118 136]


In [8]:
len(aqi_ndarray)

120

### EEMD分解 
#### ref：https://www.cnblogs.com/xtu-zzg/p/13153074.html

In [9]:
# 设置信号长度
t = np.arange(1,len(aqi_ndarray)+1)

# 提取imfs和剩余信号res
emd = EMD()
emd.emd(aqi_ndarray)
imfs, res = emd.get_imfs_and_residue()

### 制图

In [10]:
# 绘制 IMF
vis = Visualisation()
#vis.plot_imfs(imfs=imfs, residue=res, t=t, include_residue=True)  # vis.plot_imfs(imfs=imfs, residue=res, t=t, include_residue=True)

# 绘制并显示所有提供的IMF的瞬时频率
#vis.plot_instant_freq(t, imfs=imfs)
# vis.show()

### 输出
#### 输出每一个IMF分量 和 Res
##### 用于后续lsmt

In [11]:
# 查验size

print(len(imfs))
print(len(imfs[1]))
print(len(res))

5
120
120


In [12]:
# 分别输出为DF
aqi_imfs_out = pd.DataFrame(imfs).T
aqi_res_out  = pd.DataFrame(res)  # 无需转置

# 合并 
aqi_eemd_out = pd.concat([aqi_imfs_out,aqi_res_out], axis=1)  # 横向合并

# 数据展示
aqi_eemd_out.head(5)

,0,1,2,3,4,0
0,-29.022702,-18.148571,38.989552,-28.918025,-30.597601,148.697348
1,36.411221,-23.304394,43.721009,-29.767730,-30.772997,148.712890
2,-32.085554,-27.880672,45.303586,-29.161563,-30.904735,148.728939
3,25.970331,-10.838060,43.110231,-26.999407,-30.988580,148.745486
4,-28.648378,17.328720,36.024540,-23.447929,-31.019473,148.762521


In [13]:
# 修改列名
columns_list = []
for i in range(0, len(imfs)):
    # print(type(i))
    imfs_order = i+1
    columns_list.append('IMF%s' % imfs_order)
columns_list.append('Res')
# print(columns_list)
aqi_eemd_out.columns = columns_list

# 修改 index_col
aqi_eemd_out.index = data_0.index

In [14]:
aqi_eemd_out.head(5)

,IMF1,IMF2,IMF3,IMF4,IMF5,Res
日期,,,,,,
2014-01-01,-29.022702,-18.148571,38.989552,-28.918025,-30.597601,148.697348
2014-01-02,36.411221,-23.304394,43.721009,-29.767730,-30.772997,148.712890
2014-01-03,-32.085554,-27.880672,45.303586,-29.161563,-30.904735,148.728939
2014-01-04,25.970331,-10.838060,43.110231,-26.999407,-30.988580,148.745486
2014-01-05,-28.648378,17.328720,36.024540,-23.447929,-31.019473,148.762521


### 输出excel

In [15]:
output_path= 'C:\\Users\\iii\\Desktop\\LSTM+AQI\\数据和测算结果\\'
aqi_eemd_out.to_excel(output_path+'aqi_eemd_out.xlsx')

In [16]:
# 其他测试 无关程序
aqi_eemd_out.index

DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2014-04-21', '2014-04-22', '2014-04-23', '2014-04-24',
               '2014-04-25', '2014-04-26', '2014-04-27', '2014-04-28',
               '2014-04-29', '2014-04-30'],
              dtype='datetime64[ns]', name='日期', length=120, freq=None)